# Installation

clone the repository


In [ ]:
!git clone https://github.com/alessiobenincasa/pdf-contact-api.git

Cloning into 'pdf-contact-api'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 61 (delta 22), reused 51 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 26.30 KiB | 3.29 MiB/s, done.
Resolving deltas: 100% (22/22), done.


change directory, libraries and install necessary dependencies

In [ ]:
%%bash
#change directory
cd pdf-contact-api

# (Optional) Create and activate a virtual environment - note this may not persist in Colab.
# Remove problematic reportlab version from requirements.txt (if necessary)
sed -i '/reportlab==3\.6\.4/d' requirements.txt

# Install dependencies.
pip install -r requirements.txt

# pip install ngrox
pip install pyngrok

# Install a compatible version of reportlab (if needed)
pip install reportlab==3.6.12

# Install llama-cpp-python.
pip install llama-cpp-python

echo "Colab setup complete. You can now run the API or test the LLM functionality."




  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.2 MB/s eta 0:00:00
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31668 sha256=f3099964b581e18ae5cd6278250076647e4cab998f7b58f7c16286b93e000c48
  Stored in directory: /root/.cache/pip/wheels/3a/d4/d0/dbbf3c534e9c5dfbdb2729d10bd12548b8a07103030c07d9e7
Successfully built python-multipart
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.6
    Uninstalling pydantic-2.10.6:
      Successfully uninstalled pydantic-2.10.6
Colab setup complete. You can

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.35 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.21 which is incompatible.
albumentations 2.0.4 requires pydantic>=2.9.2, but you have pydantic 1.10.21 which is incompatible.
google-genai 0.8.0 requires pydantic<3.0.0dev,>=2.0.0, but you have pydantic 1.10.21 which is incompatible.
sigstore-rekor-types 0.0.18 requires pydantic[email]<3,>=2, but you have pydantic 1.10.21 which is incompatible.
wandb 0.19.6 requires pydantic<3,>=2.6, but you have pydantic 1.10.21 which is incompatible.
langchain 0.3.18 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
sigstore 3.6.1 requires pydantic<3,>=2, but you have pydantic 1.10.21 which is incompatible.


In [ ]:
!pwd

/content


## Refactoring of llm.py and adding cli.py

create a file in pdf-contact-api/app named cli.py containing the following code :

In [ ]:
#!/usr/bin/env python
import sys
import os

# Debug output: show current working directory and sys.path before modifying it.
print("Current working directory:", os.getcwd())
print("Initial sys.path:", sys.path)

# Insert the current working directory (repository root) at the beginning of sys.path.
sys.path.insert(0, os.getcwd())

# Debug output: show sys.path after modification.
print("Updated sys.path:", sys.path)

import argparse
import subprocess
import time

def launch_server():
    """
    Launch the FastAPI server using uvicorn in a subprocess.
    The server is defined in app/main.py.
    """
    print("Launching API server...")
    proc = subprocess.Popen(
        [sys.executable, "-m", "uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000", "--reload"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    time.sleep(5)
    print("Server should be up now at http://localhost:8000")
    print("Press Ctrl+C to stop the server.")
    try:
        proc.wait()
    except KeyboardInterrupt:
        print("\nTerminating server...")
        proc.terminate()
        proc.wait()

def test_llm(prompt: str):
    """
    Test the LLM extraction function directly using the provided prompt.
    """
    print("Testing LLM extraction function...")
    try:
        from app.llm import extract_contacts_with_hf
        print("Successfully imported app.llm!")
    except ImportError as e:
        print("Error importing LLM module. Make sure your PYTHONPATH is set correctly.")
        print("ImportError:", e)
        sys.exit(1)
    output = extract_contacts_with_hf(prompt)
    print("\n=== LLM Output ===")
    print(output)
    print("==================\n")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="CLI for launching and testing the PDF Uploader API."
    )
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument(
        "--launch",
        action="store_true",
        help="Launch the API server."
    )
    group.add_argument(
        "--test",
        action="store_true",
        help="Run a test of the LLM extraction function."
    )
    parser.add_argument(
        "--prompt",
        type=str,
        default="Contact John Doe at john@example.com or call 123-456-7890.",
        help="Prompt text for testing the LLM extraction."
    )

    args = parser.parse_args()

    if args.launch:
        launch_server()
    elif args.test:
        test_llm(args.prompt)



modify llm.py and add instead :

In [ ]:
# app/llm.py
from llama_cpp import Llama

# Instantiate the model using the from_pretrained method.
llm = Llama.from_pretrained(
    repo_id="TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF",
    filename="capybarahermes-2.5-mistral-7b.Q2_K.gguf",
)

def extract_contacts_with_hf(text: str, max_tokens: int = 128, temperature: float = 0.7, top_p: float = 0.95) -> str:
    """
    Uses the CapybaraHermes chat model to extract contact information from the given text.

    The prompt instructs the model to extract emails, phone numbers, addresses, etc.
    It returns a JSON formatted string with the extracted contacts.
    """
    prompt = (
        "Extract all the contact information (emails, phone numbers, addresses, etc.) "
        "from the following text and return it in JSON format:\n\n"
        f"{text}\n\n"
        "JSON:"
    )
    try:
        # Use create_chat_completion with a single user message.
        response = llm.create_chat_completion(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
        )
        # The expected response structure is a dict with a 'choices' key.
        response_text = response.get("choices", [{}])[0].get("message", {}).get("content", "").strip()
        return response_text
    except Exception as e:
        print("Error during LLM query:", e)
        return "{}"

if __name__ == "__main__":
    # Multiple sample tests with varying complexity:
    sample_tests = [
        (
            "Simple Test",
            "Contact John Doe at john@example.com or call 123-456-7890. "
            "Office: 987-654-3210, Email: office@example.com."
        ),
        (
            "Moderate Test",
            "For inquiries, please reach out to Alice Smith at alice.smith@company.com, "
            "Bob Jones at bob.jones@firm.org, or visit our website. Alternatively, call our "
            "support hotline at (555) 123-4567."
        ),
        (
            "Complex Test",
            "Hello, please contact our sales team: John Doe, email: sales@example.com, phone: +44 20 7946 0958; "
            "for support, contact Jane Roe at jane.roe@support.co.uk or call (800) 555-1212 ext. 99. Also, for HR queries, "
            "reach out at hr@ourcompany.com. Our offices are located at 123 Business Rd., Business City, BC 12345."
        )
    ]

    # Run each test and print the extracted contacts.
    for test_name, text in sample_tests:
        print(f"--- {test_name} ---")
        contacts = extract_contacts_with_hf(text)
        print("Extracted contacts:", contacts)
        print("")


launch and test the app with the following cli :

In [ ]:
!python app/cli.py --test --prompt "For contact extraction, please process the following information: Our company contacts are as follows: Jane Doe (Email: jane.doe@example.com, Phone: (555) 987-6543, Address: 123 Main St, Springfield); John Smith (Email: john.smith@company.com, Phone: (555) 234-5678, Address: 456 Oak Ave, Metropolis); and Alex Johnson (Email: alex.johnson@enterprise.org, Phone: (555) 345-6789, Address: 789 Pine Rd, Gotham). For urgent issues, contact our support hotline at (555) 111-2222 or email hotline@company.com. Return the extracted details in JSON format."


Current working directory: /content/pdf-contact-api
Initial sys.path: ['/content/pdf-contact-api/app', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages']
Updated sys.path: ['/content/pdf-contact-api', '/content/pdf-contact-api/app', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages']
Testing LLM extraction function...
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--CapybaraHermes-2.5-Mistral-7B-GGUF/snapshots/8bea614edd9a2d5d9985a6e6c1ecc166261cacb8/./capybarahermes-2.5-mistral-7b.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                    